# Selection Functions
## Depth maps and selection functions

The simplest selection function available is the field MOC which specifies the area for which there is Herschel data. Each pristine catalogue also has a MOC defining the area for which that data is available.

This notebook should determine the correct field and filenames based on being placed in the correct folder.

The next stage is to provide mean flux standard deviations which act as a proxy for the catalogue's 5$\sigma$ depth

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100) [with local modifications]
This notebook was executed on: 
2020-12-01 11:55:20.600299


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
import numpy as np
from pymoc import MOC
import healpy as hp
#import pandas as pd #Astropy has group_by function so apandas isn't required.
import seaborn as sns
import glob
import gc
import warnings
#We ignore warnings - this is a little dangerous but a huge number of warnings are generated by empty cells later
warnings.filterwarnings('ignore')

from herschelhelp_internal.utils import inMoc, coords_to_hpidx, flux_to_mag
from herschelhelp_internal.masterlist import find_last_ml_suffix, nb_ccplots

from astropy.io.votable import parse_single_table
import yaml

In [3]:
def make_depth_map(meta_yml):
    meta = yaml.load(open(meta_yml, 'r'))
    FIELD = meta['field']
    #FILTERS_DIR = "/Users/rs548/GitHub/herschelhelp_python/database_builder/filters/"
    FILTERS_DIR = "/opt/herschelhelp_python/database_builder/filters/"

    OUT_DIR = meta_yml.replace('meta_main.yml','data/')
    #SUFFIX = find_last_ml_suffix()
    SUFFIX = meta['final'].split('/')[-1].split('_')[-1].strip('.fits')

    master_catalogue_filename = meta['final'].split('/')[-1]
    master_catalogue = Table.read("{}/{}".format(OUT_DIR, master_catalogue_filename))

    for col in master_catalogue.colnames:
        if  (
            col.startswith('m')
            or col.startswith('flag')
            or col =='redshift'
            or col=='zspec'
            
        ):
            master_catalogue.remove_column(col)
    
    print("Depth maps produced using: {}".format(master_catalogue_filename))

    ORDER = 10
  

    field_moc = MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(FIELD))
    # Remove sources whose signal to noise ratio is less than five as these will have been selected using forced 
    # photometry and so the errors will not refelct the RMS of the map 
    # For xid+ saource use s/n >2 as that is criteria we use for CIGALE
    for n,col in enumerate(master_catalogue.colnames):
        if col.startswith("f_spire") or col.startswith("f_pacs")or col.startswith("f_mips"):
            err_col = "ferr{}".format(col[1:])
            errs = master_catalogue[err_col]
            fluxes = master_catalogue[col]
            mask = fluxes/errs < 2.0
            master_catalogue[col][mask] = np.nan
            master_catalogue[err_col][mask] = np.nan
        elif col.startswith("f_"):
            err_col = "ferr{}".format(col[1:])
            errs = master_catalogue[err_col]
            fluxes = master_catalogue[col]
            mask = fluxes/errs < 5.0
            master_catalogue[col][mask] = np.nan
            master_catalogue[err_col][mask] = np.nan
            
            
    #Add a column to the catalogue with the order=ORDER hp_idx
    master_catalogue.add_column(Column(data=coords_to_hpidx(master_catalogue['ra'],
                                                       master_catalogue['dec'],
                                                       ORDER), 
                                   name="hp_idx_O_{}".format(str(ORDER))
                                  )
                           )
    # Convert catalogue to pandas and group by the order=ORDER pixel
    group = master_catalogue.group_by(["hp_idx_O_{}".format(str(ORDER))])
    depths = Table()
    depths['hp_idx_O_13'] = list(field_moc.flattened(13))
    depths.add_column(Column(data=hp.pixelfunc.ang2pix(2**ORDER,
                     hp.pixelfunc.pix2ang(2**13, depths['hp_idx_O_13'], nest=True)[0],
                     hp.pixelfunc.pix2ang(2**13, depths['hp_idx_O_13'], nest=True)[1],
                     nest = True),
                     name="hp_idx_O_{}".format(str(ORDER))
                        )
                 )
    for col in master_catalogue.colnames:
        if col.startswith("f_"):
            errcol = "ferr{}".format(col[1:])
            depths = join(depths, 
                      group["hp_idx_O_{}".format(str(ORDER)), errcol].groups.aggregate(np.nanmean),
                     join_type='left')
            depths[errcol].name = errcol + "_mean"
            depths = join(depths, 
                      group["hp_idx_O_{}".format(str(ORDER)), col].groups.aggregate(lambda x: np.nanpercentile(x, 90.)),
                     join_type='left')
            depths[col].name = col + "_p90"
            
    standard = ['u', 'g', 'r', 'i', 'z', 'y', 'j', 'h', 'k', 'ks']
    tot_bands = [column[2:] for column in master_catalogue.colnames 
             if (column.startswith('f_') & ~column.startswith('f_ap_'))]
    ap_bands = [column[5:] for column in master_catalogue.colnames 
            if column.startswith('f_ap_') ]
    bands = set(tot_bands) | set(ap_bands)
    bands
    
    for col in depths.colnames:
        if depths[col].dtype == 'float64' or depths[col].dtype == 'float32':
            depths[col].fill_value = np.nan
        
    depths = depths.filled()
    
    for band in standard:
        print(band)
        #ap_bands
        list_of_ap_cols = np.array([ b if b.endswith('_{}'.format(band)) else None for b in ap_bands])
        list_of_ap_cols = list_of_ap_cols[list_of_ap_cols != None]
        list_of_ap_cols = ['ferr_{}_mean'.format(b) for b in list_of_ap_cols]
        print(list_of_ap_cols)
        if len(list_of_ap_cols) >0:
            depths['ferr_ap_{}_mean'.format(band)]  = np.nanmin([depths[t] for t in list_of_ap_cols], axis=0)
        #tot_bands
        list_of_tot_cols = np.array([ b if b.endswith('_{}'.format(band)) else None for b in tot_bands])
        list_of_tot_cols = list_of_tot_cols[list_of_tot_cols != None]
        list_of_tot_cols = ['ferr_{}_mean'.format(b) for b in list_of_tot_cols]
        print(list_of_tot_cols)
        if len(list_of_tot_cols) >0:
            depths['ferr_{}_mean'.format(band)]  = np.nanmin([depths[t] for t in list_of_tot_cols], axis=0)
    depths.write(
        "../dmu32_{}/data/depths_{}_{}.fits".format(
            FIELD.replace('HATLAS-',''), 
            FIELD.lower(), 
            SUFFIX), 
        overwrite=True
    )

In [4]:
meta_files = glob.glob('../*/meta_main.yml')

In [5]:
meta_files

['../dmu32_SA13/meta_main.yml',
 '../dmu32_Bootes/meta_main.yml',
 '../dmu32_xFLS/meta_main.yml',
 '../dmu32_GAMA-15/meta_main.yml',
 '../dmu32_GAMA-12/meta_main.yml',
 '../dmu32_HDF-N/meta_main.yml',
 '../dmu32_NGP/meta_main.yml',
 '../dmu32_ELAIS-N2/meta_main.yml',
 '../dmu32_EGS/meta_main.yml',
 '../dmu32_COSMOS/meta_main.yml',
 '../dmu32_SGP/meta_main.yml',
 '../dmu32_Lockman-SWIRE/meta_main.yml',
 '../dmu32_XMM-LSS/meta_main.yml',
 '../dmu32_ELAIS-N1/meta_main.yml',
 '../dmu32_AKARI-SEP/meta_main.yml',
 '../dmu32_GAMA-09/meta_main.yml',
 '../dmu32_AKARI-NEP/meta_main.yml',
 '../dmu32_ELAIS-S1/meta_main.yml',
 '../dmu32_XMM-13hr/meta_main.yml',
 '../dmu32_Herschel-Stripe-82/meta_main.yml',
 '../dmu32_CDFS-SWIRE/meta_main.yml',
 '../dmu32_SPIRE-NEP/meta_main.yml',
 '../dmu32_SSDF/meta_main.yml']

In [6]:
failures = set()
for m in meta_files:
    try:
        make_depth_map(m)
    except:
        print(m, 'failed')
        failures.add(m)
    gc.collect()
    time.sleep(10)
failures

Depth maps produced using: SA13_20180501.fits
u
[]
[]
g
['ferr_90prime_g_mean']
['ferr_90prime_g_mean']
r
['ferr_90prime_r_mean']
['ferr_90prime_r_mean']
i
[]
[]
z
['ferr_mosaic_z_mean']
['ferr_mosaic_z_mean']
y
[]
[]
j
['ferr_ukidss_j_mean']
['ferr_ukidss_j_mean']
h
[]
[]
k
[]
[]
ks
[]
[]
Depth maps produced using: Bootes_20190701.fits
u
['ferr_lbc_u_mean']
['ferr_lbc_u_mean']
g
['ferr_90prime_g_mean', 'ferr_gpc1_g_mean']
['ferr_90prime_g_mean', 'ferr_gpc1_g_mean']
r
['ferr_gpc1_r_mean', 'ferr_90prime_r_mean', 'ferr_mosaic_r_mean']
['ferr_gpc1_r_mean', 'ferr_90prime_r_mean', 'ferr_mosaic_r_mean']
i
['ferr_gpc1_i_mean', 'ferr_mosaic_i_mean']
['ferr_gpc1_i_mean', 'ferr_mosaic_i_mean']
z
['ferr_gpc1_z_mean', 'ferr_suprime_z_mean', 'ferr_mosaic_z_mean', 'ferr_90prime_z_mean']
['ferr_gpc1_z_mean', 'ferr_suprime_z_mean', 'ferr_mosaic_z_mean', 'ferr_90prime_z_mean']
y
['ferr_gpc1_y_mean', 'ferr_lbc_y_mean']
['ferr_gpc1_y_mean', 'ferr_lbc_y_mean']
j
['ferr_ukidss_j_mean', 'ferr_newfirm_j_mean

{'../dmu32_NGP/meta_main.yml', '../dmu32_SGP/meta_main.yml'}